<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/automatically_map_internal_searches_to_landing_pages/automatically_map_internal_searches_to_landing_pages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Script by @LeeFootSEO
**Early Access To Apps & Feature Requests:** https://www.patreon.com/leefootseo

**Buy me a Coffee:** https://www.buymeacoffee.com/leefootseo

#Internal Search Report Mapper V2

What This Script Does

Takes the Google Analytics search terms report and merges with a Screaming Frog crawl file to find opportunities to map internal searches to.

#Input Files

1.   Search Terms Report - (Analytics > Behaviour > Site Search > Search Terms)
2.   Screaming Frog Crawl - Internal_html.csv

In [ ]:
!pip install polyfuzz
!pip install pandas

     |████████████████████████████████| 14.5MB 261kB/s 
     |████████████████████████████████| 757kB 36.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [ ]:
from polyfuzz import PolyFuzz
from glob import glob
import pandas as pd

In [ ]:
#Use to import io, which opens the file from the Colab drive
from google.colab import files 
import io

In [ ]:
# Upload Screaming Frog Crawl File 
uploaded = files.upload()

Saving internal_html.csv to internal_html (3).csv


In [ ]:
 # Now use the below code to read the csv in pandas' dataframe
sf_filename=list(uploaded.keys())[0]
try:
  df_sf = pd.read_csv(io.StringIO(uploaded[sf_filename].decode('utf-8', errors='ignore')))[["H1-1", "Address", "Indexability"]]
except Exception:
  print('Make sure your Screaming Frog Crawl contains the "Address", "H1-1" and "Indexability Columns')
print(df_sf.head())

                          H1-1  ...   Indexability
0  Internal and external doors  ...      Indexable
1                          NaN  ...  Non-Indexable
2          Frosted Glass Doors  ...  Non-Indexable
3         Wenge Internal Doors  ...      Indexable
4             Fire Rated Locks  ...      Indexable

[5 rows x 3 columns]


# Upload Site Search Report from Google Analytics (Must be Excel File Export!)
### Behaviour > Site Search > Search Terms

In [ ]:
# Upload Internal Site Searches Report from Google Analytics (Excel File)
uploaded = files.upload()

Saving Analytics DOS - MV - Annotations Search Terms 20200608-20210608.xlsx to Analytics DOS - MV - Annotations Search Terms 20200608-20210608.xlsx


In [ ]:
ga_filename=list(uploaded.keys())[0]  # get the filename from the upload
df_ga = pd.read_excel(ga_filename, 'Dataset1')  # choose the right sheet from Excel
print(df_ga.head())

  Search Term  Total Unique Searches  ...  Time After Search  Avg. Search Depth
0      bifold                   1101  ...         771.655767           3.164396
1     suffolk                    914  ...         981.705689           4.935449
2       white                    730  ...         757.075342           2.835616
3     cottage                    633  ...         799.952607           3.706161
4         oak                    596  ...         541.080537           2.953020

[5 rows x 7 columns]


In [ ]:
df_ga['word_count'] = df_ga['Search Term'].str.count(' ') +1
df_ga['word_count'] = df_ga['word_count'].dropna()
df_ga =df_ga[df_ga["word_count"].notna()]
df_ga['word_count'] = df_ga['word_count'].astype(int)
df_ga = df_ga[df_ga['word_count'] >= 2]
df_ga

,Search Term,Total Unique Searches,Results Page Views/Search,% Search Exits,% Search Refinements,Time After Search,Avg. Search Depth,word_count
6,solid core,569,3.388401,0.523726,0.036307,845.427065,2.500879,2
10,4 panel,442,2.585973,0.273756,0.167104,462.054299,2.423077,2
11,pattern 10,433,2.378753,0.145497,0.159223,995.715935,3.605081,2
12,Double doors,408,1.926471,0.289216,0.234097,258.428922,2.561275,2
13,6 panel,384,2.458333,0.263021,0.164195,622.255208,2.986979,2
...,...,...,...,...,...,...,...,...
492,cottage oak,41,2.512195,0.219512,0.126214,635.463415,2.853659,2
493,door furniture,41,2.341463,0.073171,0.250000,683.390244,3.658537,2
494,Door Pair,41,3.878049,0.195122,0.182390,214.731707,2.000000,2
495,External Stable Door,41,3.073171,0.268293,0.206349,505.682927,3.243902,3


In [ ]:
# clean the data
df_sf["H1-1"] = df_sf["H1-1"].str.lower()
df_ga["Search Term"] = df_ga[
    "Search Term"
].str.lower()  # convert to lower case for matching

# group search terms together
df_ga = (df_ga.groupby("Search Term").agg({"Total Unique Searches": "sum", "Results Page Views/Search": "mean", "% Search Exits": "mean","% Search Refinements": "mean", "Time After Search":"mean","Avg. Search Depth":"mean","word_count":"first"	}).reset_index())


try:
    # drop non-indexable pages
    df_sf = df_sf[~df_sf["Indexability"].isin(["Non-Indexable"])]
except Exception:
    pass

df_ga = df_ga[df_ga["Search Term"].notna()]  # keep rows which are not NaN
df_sf = df_sf[df_sf["H1-1"].notna()]  # keep rows which are not NaN

In [ ]:
# create lists from dfs
ga_list = list(df_ga["Search Term"])
sf_list = list(df_sf["H1-1"])

Do the matching!

In [ ]:
# instantiate PolyFuzz model, choose TF-IDF as the similarity measure and match the two lists.
model = PolyFuzz("TF-IDF").match(ga_list, sf_list)

# make the polyfuzz dataframe
df_pf_matches = model.get_matches()

# keep only rows which are not NaN
df_pf_matches = df_pf_matches[df_pf_matches["To"].notna()]

# merge original ga search term data back into polyfuzz df
df_pf_matches_df_ga = pd.merge(
    df_pf_matches, df_ga, left_on="From", right_on="Search Term", how="inner"
)

# create final_df + merge original screaming frog data back in
final_df = pd.merge(df_pf_matches_df_ga, df_sf, left_on="To", right_on="H1-1")

In [ ]:
# sort by opportunity
final_df = final_df.sort_values(by="Total Unique Searches", ascending=False)

# Round Float to two decimal places
final_df = final_df.round(decimals=2)

# rename the cols
final_df.rename(
    columns={"From": "Search Term", "To": "Matched H1", "Address": "Matched URL"},
    inplace=True,
)

# drop the duplicate Search Terms Column
#final_df.drop(final_df.columns[4], axis=1, inplace=True)
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

In [ ]:
# set new column order for final df
cols = [
    "Search Term",
    "Matched H1",
    "Similarity",
    "Matched URL",
    "Total Unique Searches",
    "Results Page Views/Search",
    "% Search Exits",
    "% Search Refinements",
    "Time After Search",
    "Avg. Search Depth",
]

# re-index columns into a logical order
final_df = final_df.reindex(columns=cols)

In [ ]:
# drop duplicate keywords
final_df.drop_duplicates(subset=["Search Term"], inplace=True)

In [ ]:
# export the final exact and partial dataframes
final_df_exact = final_df.loc[final_df["Similarity"] == 1]
final_df_partial = final_df.loc[final_df["Similarity"] != 1].copy()

final_df_partial.sort_values(
    ["Similarity", "Total Unique Searches"],
    ascending=[False, False],
    inplace=True,
)
print(final_df_exact.head())
print(final_df_partial.head())
print(final_df.head())

        Search Term      Matched H1  ...  Time After Search Avg. Search Depth
140    french doors    french doors  ...             368.49              2.01
41   internal doors  internal doors  ...             388.17              2.89
213   sliding doors   sliding doors  ...             443.81              2.48
65     door handles    door handles  ...             604.91              3.13
123  external doors  external doors  ...             445.31              2.37

[5 rows x 10 columns]
                Search Term  ... Avg. Search Depth
212            sliding door  ...              2.03
134             stable door  ...              2.67
70             cottage door  ...              3.10
139             french door  ...              3.31
131  external hardwood door  ...              3.69

[5 rows x 10 columns]
             Search Term  ... Avg. Search Depth
125  external door frame  ...              3.24
115         double doors  ...              3.18
212         sliding door  ...       

In [ ]:
# make and download the csv files
final_df_exact.to_csv("Exact Matches.csv", index=False)
final_df_partial.to_csv("Partial Matches.csv", index=False)
final_df.to_csv("All Matches.csv", index=False)

#Download the csv files
Chrome may ask if you would like to download multiple files at once before downloading. Remember to click the pop up!

In [ ]:
# export all exact matches
try:
  files.download("Exact Matches.csv")
except FileNotFoundError:
  print("No Exact Matches to Export!")
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# export all partial matches
try:
  files.download("Partial Matches.csv")
except FileNotFoundError:
  print("No Partial Matches to Export!")
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# export all matches
try:
  files.download("All Matches.csv")
except FileNotFoundError:
  print("No Matches to Export!")
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>